<a href="https://colab.research.google.com/github/yoheikikuta/US-patent-analysis/blob/master/colab/BERT_pretrain_with_patent_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT pretraining with patent data

In [0]:
from google.colab import auth
auth.authenticate_user()

## Data preparation

In [2]:
!gsutil cp gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-info/citations_info_3000+3000.df.gz ./

!gsutil cp gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/training_app_3000.df.gz ./  
!gsutil cp gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/testset_app_3000.df.gz ./
!gsutil cp gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/grants_for_3000+3000.df.gz ./

Copying gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-info/citations_info_3000+3000.df.gz...
/ [1 files][506.5 KiB/506.5 KiB]                                                
Operation completed over 1 objects/506.5 KiB.                                    
Copying gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/training_app_3000.df.gz...
| [1 files][ 45.0 MiB/ 45.0 MiB]                                                
Operation completed over 1 objects/45.0 MiB.                                     
Copying gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/testset_app_3000.df.gz...
\ [1 files][ 45.5 MiB/ 45.5 MiB]                                                
Operation completed over 1 objects/45.5 MiB.                                     
Copying gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/grants_for_3000+3000.df.gz...
- [1 files][129.4 MiB/129.4 MiB]                                                
Operation completed over 1 objects/129.4 MiB.           

In [0]:
import pandas as pd
import numpy as np

In [0]:
citations_info_target = pd.read_pickle("./citations_info_3000+3000.df.gz")
test_app = pd.read_pickle("./testset_app_3000.df.gz")
grants = pd.read_pickle("./grants_for_3000+3000.df.gz")
train_app = pd.read_pickle("./training_app_3000.df.gz")

In [5]:
train_app.head(3)

,app_id,xml
0,12130785,"<us-patent-application lang=""EN"" dtd-version=""..."
1,12652424,"<us-patent-application lang=""EN"" dtd-version=""..."
2,12214532,"<us-patent-application lang=""EN"" dtd-version=""..."


In [0]:
import re


CLAIM_PAT = re.compile(r'<claims[^>]*>(.*)</claims>',re.MULTILINE|re.DOTALL)
TAG_PAT = re.compile(r"<.*?>")
LB_PAT = re.compile(r'[\t\n\r\f\v][" "]*')
CANCELED_PAT = re.compile(r'[0-9]+.*\. \(canceled\)[" "]')
NUM_PAT = re.compile(r'[" "]?[0-9]+[" "]?\.[" "]?')


def whole_xml_to_claim_xml(whole):
    mat = CLAIM_PAT.search(whole)
    return mat.group(1)


def whole_xml_to_claim(whole):
    return TAG_PAT.sub(' ', whole_xml_to_claim_xml(whole))


def remove_linebreak_from_claim(claim):
    return LB_PAT.sub('', claim)


def remove_canceled_claim(claim):
    return CANCELED_PAT.sub('', claim)


def remove_claim_numbers(claim):
    return NUM_PAT.sub('', claim)  

Test data will NOT be used for pretraining.

In [7]:
%%time

train_app["claim_app"] = train_app["xml"].map(whole_xml_to_claim).map(remove_canceled_claim).map(remove_claim_numbers).map(remove_linebreak_from_claim)
train_app = train_app.drop("xml", axis=1)
train_app.head()

# test_app["claim_app"] = test_app["xml"].map(whole_xml_to_claim).map(remove_canceled_claim).map(remove_claim_numbers).map(remove_linebreak_from_claim)
# test_app = test_app.drop("xml", axis=1)
# test_app.head()

grants["claim_cited_grant"] = grants["xml"].map(whole_xml_to_claim).map(remove_canceled_claim).map(remove_claim_numbers).map(remove_linebreak_from_claim)
grants = grants.drop("xml", axis=1)
grants.head()

CPU times: user 7.93 s, sys: 188 ms, total: 8.11 s
Wall time: 8.12 s


In [8]:
train_app.head(3)

,app_id,claim_app
0,12130785,A system for differentiating noise from an arr...
1,12652424,A method of allocating resources in a data war...
2,12214532,A controlling method of a media processing app...


In [0]:
test = train_app['claim_app'][0]

In [16]:
test.replace(".", "\n")

'A system for differentiating noise from an arrhythmia of a heart, comprising:a noise discriminator configured to receive an electrocardiogram (EGM) signal and to discriminate between an organized EGM signal and a chaotic EGM signal based at least in part on an impedance parameter associated with a lead that provides an electrical connection to the heart; a signal analyzer configured to determine whether a chaotic signal is caused by a disturbance in the lead\n The system of  claim 1 , further comprising a high voltage delivery system configured to deliver a high voltage therapy signal to the heart if the EGM signal is an organized signal\n The system of  claim 2 , further comprising a high voltage confirmation system configured to adjust or terminate the high voltage therapy based on the impedance parameter\n The system of  claim 3 , wherein the signal analyzer is part of the high voltage confirmation system\n The system of  claim 3 , wherein the lead comprises a high voltage lead for

In order to make a pretraining data, use simple preprocessing: replacing a (period + space) with (period + line break).

In [0]:
with open("./test.txt", "w+") as f:
    f.write(test.replace(". ", ".\n").lower())

In [38]:
!cat ./test.txt

a system for differentiating noise from an arrhythmia of a heart, comprising:a noise discriminator configured to receive an electrocardiogram (egm) signal and to discriminate between an organized egm signal and a chaotic egm signal based at least in part on an impedance parameter associated with a lead that provides an electrical connection to the heart; a signal analyzer configured to determine whether a chaotic signal is caused by a disturbance in the lead.
the system of  claim 1 , further comprising a high voltage delivery system configured to deliver a high voltage therapy signal to the heart if the egm signal is an organized signal.
the system of  claim 2 , further comprising a high voltage confirmation system configured to adjust or terminate the high voltage therapy based on the impedance parameter.
the system of  claim 3 , wherein the signal analyzer is part of the high voltage confirmation system.
the system of  claim 3 , wherein the lead comprises a high voltage lead for deli

In [29]:
len(pd.concat([train_app['claim_app'], grants['claim_cited_grant']]))

9440

In [39]:
%%time

with open("./training_data.txt", "w+") as f:
    for one_stuff in pd.concat([train_app['claim_app'], grants['claim_cited_grant']]):
        f.write(one_stuff.replace(". ", ".\n").lower())

CPU times: user 214 ms, sys: 101 ms, total: 316 ms
Wall time: 325 ms


## Create training data for BERT

In [5]:
!git clone https://github.com/google-research/bert.git

fatal: destination path 'bert' already exists and is not an empty directory.


In [8]:
!gsutil cp -r gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12 ./

Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/bert_config.json...
Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001...
Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/bert_model.ckpt.index...
Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/bert_model.ckpt.meta...
\ [4 files][420.9 MiB/420.9 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/checkpoint...
Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncas

In [36]:
!ls

adc.json			testset_app_3000.df.gz
bert				test.txt
citations_info_3000+3000.df.gz	training_app_3000.df.gz
grants_for_3000+3000.df.gz	training_data.txt
sample_data			uncased_L-12_H-768_A-12


In [6]:
%cd bert

/content/bert


In [41]:
!ls

CONTRIBUTING.md		    predicting_movie_reviews_with_bert_on_tf_hub.ipynb
create_pretraining_data.py  README.md
extract_features.py	    requirements.txt
__init__.py		    run_classifier.py
LICENSE			    run_classifier_with_tfhub.py
modeling.py		    run_pretraining.py
modeling_test.py	    run_squad.py
multilingual.md		    sample_text.txt
optimization.py		    tokenization.py
optimization_test.py	    tokenization_test.py


In [42]:
%%time

!python create_pretraining_data.py \
  --input_file=../training_data.txt \
  --output_file=../training_data.tfrecord \
  --vocab_file=../uncased_L-12_H-768_A-12/vocab.txt \
  --do_lower_case=True \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=12345 \
  --dupe_factor=5

W0811 11:48:29.829970 140302422521728 deprecation_wrapper.py:119] From create_pretraining_data.py:469: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0811 11:48:29.830970 140302422521728 deprecation_wrapper.py:119] From create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0811 11:48:29.831152 140302422521728 deprecation_wrapper.py:119] From create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0811 11:48:29.831310 140302422521728 deprecation_wrapper.py:119] From /content/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0811 11:48:29.946908 140302422521728 deprecation_wrapper.py:119] From create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W0811 11:48:29.953678 140302422521728 deprecation_wrapper.p

In [43]:
# About 350 [MiB]

!stat -c %s ../training_data.tfrecord

368197714


In [65]:
!gsutil cp ../training_data.tfrecord gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-pretrain-BERT/

Copying file://../training_data.tfrecord [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

-
Operation completed over 1 objects/351.1 MiB.                                    


## Pretraining

In [2]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.50.41.114:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 7060222105455313451),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 8716928994900460621),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 13969103642174635354),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 7325847632783373539),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 1671794254798757949),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 11964686727760805423),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 4947439867643438348),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 5839443967131183485),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 62606776377221

W0811 22:50:50.996374 140083162711936 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



NOTE: need to give access rights to INIT_CKPT GCS to cloud TPUs.

In [0]:
INPUT_FILE = "gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-pretrain-BERT/training_data.tfrecord"
OUTPUT_GCS = "gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-pretrain-BERT"
INIT_CKPT = "gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/bert_model.ckpt"

In [14]:
%%time

!python run_pretraining.py \
  --input_file={INPUT_FILE} \
  --output_dir={OUTPUT_GCS} \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --num_tpu_cores=8 \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=../uncased_L-12_H-768_A-12/bert_config.json \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=70000 \
  --num_warmup_steps=1000 \
  --learning_rate=5e-5
#  --init_checkpoint={INIT_CKPT} \  # Only need to add at the first training time.

W0812 03:07:13.807877 140024347654016 deprecation_wrapper.py:119] From /content/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0812 03:07:13.809203 140024347654016 deprecation_wrapper.py:119] From run_pretraining.py:493: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0812 03:07:13.810020 140024347654016 deprecation_wrapper.py:119] From run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0812 03:07:13.810220 140024347654016 deprecation_wrapper.py:119] From run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0812 03:07:13.810383 140024347654016 deprecation_wrapper.py:119] From /content/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0812 03:07:13.812720 140024347654016 deprecation_wrapper.py:119] Fro

## Memo

```
I0811 12:50:14.190979 140514602837888 run_pretraining.py:483] ***** Eval results *****
I0811 12:50:14.191113 140514602837888 run_pretraining.py:485]   global_step = 20
I0811 12:50:14.191493 140514602837888 run_pretraining.py:485]   loss = 1.0284224
I0811 12:50:14.191617 140514602837888 run_pretraining.py:485]   masked_lm_accuracy = 0.8165625
I0811 12:50:14.191753 140514602837888 run_pretraining.py:485]   masked_lm_loss = 0.86381125
I0811 12:50:14.191835 140514602837888 run_pretraining.py:485]   next_sentence_accuracy = 0.95
I0811 12:50:14.191914 140514602837888 run_pretraining.py:485]   next_sentence_loss = 0.1793103
CPU times: user 1.56 s, sys: 244 ms, total: 1.8 s
Wall time: 4min 17s
```

```
I0811 14:17:34.821915 139717559273344 run_pretraining.py:483] ***** Eval results *****
I0811 14:17:34.822047 139717559273344 run_pretraining.py:485]   global_step = 15000
I0811 14:17:34.822362 139717559273344 run_pretraining.py:485]   loss = 0.18834595
I0811 14:17:34.822492 139717559273344 run_pretraining.py:485]   masked_lm_accuracy = 0.9595625
I0811 14:17:34.822575 139717559273344 run_pretraining.py:485]   masked_lm_loss = 0.14227556
I0811 14:17:34.822673 139717559273344 run_pretraining.py:485]   next_sentence_accuracy = 0.99875
I0811 14:17:34.822756 139717559273344 run_pretraining.py:485]   next_sentence_loss = 0.0048195585
```

```
I0812 03:15:34.621796 140024347654016 run_pretraining.py:483] ***** Eval results *****
I0812 03:15:34.621907 140024347654016 run_pretraining.py:485]   global_step = 70000
I0812 03:15:34.622360 140024347654016 run_pretraining.py:485]   loss = 0.0005139347
I0812 03:15:34.622489 140024347654016 run_pretraining.py:485]   masked_lm_accuracy = 0.9999375
I0812 03:15:34.622562 140024347654016 run_pretraining.py:485]   masked_lm_loss = 0.00031679025
I0812 03:15:34.622630 140024347654016 run_pretraining.py:485]   next_sentence_accuracy = 1.0
I0812 03:15:34.622724 140024347654016 run_pretraining.py:485]   next_sentence_loss = 1.9996969e-07
```